In [22]:
import langchain
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

In [23]:
DATA_PATH= "data"
CHROMA_PATH = "chroma"

In [24]:
def load_docs():
    doc_loader = PyPDFDirectoryLoader(DATA_PATH)
    return doc_loader.load()

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [26]:
def split_documents(docs: list[Document]):
    text_splitter= RecursiveCharacterTextSplitter(
        chunk_size=800, 
        chunk_overlap=80, 
        length_function=len,
        is_separator_regex=False)
    
    return text_splitter.split_documents(docs)

In [27]:
def get_embedding_function():
    #if we use AWS
    # embeddings = BedrockEmbeddings(
    #     credentials_profile_name="default",
    #     region_name="us-east-1")
    
    #if we use Ollama 
    embeddings= OllamaEmbeddings(model="nomic-embed-text")
    
    return embeddings
    

### create the database:

In [28]:
from langchain_community.vectorstores import Chroma

In [29]:
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [30]:
def add_to_chroma(chunks: list[Document]):
    db= Chroma(persist_directory=CHROMA_PATH,
               embedding_function=get_embedding_function()
               )
    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("No new documents to add!")

In [31]:
documents = load_docs()
chunks = split_documents(documents)
add_to_chroma(chunks)

c:\Users\iraha\OneDrive - IMAM ABDULRAHMAN BIN FAISAL UNIVERSITY\Desktop\GitHub\OllamaChatbot\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


Number of existing documents in DB: 0
Adding new documents: 41


ValueError: Error raised by inference API HTTP code: 404, {"error":"model \"nomic-embed-text\" not found, try pulling it first"}